# Import libs

In [1]:
import mlflow
import pandas as pd
import numpy as np


from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.tree import DecisionTreeClassifier

# Data prep

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/maddataanalyst/for_students/master/postgraduate_pl/drzewa_decyzyjne/adult.csv")

In [3]:
data.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,earnings
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [ ]:
data.earnings = data.earnings == ">50K"

In [ ]:
data_ohe = pd.get_dummies(data)

In [ ]:
data_ohe.head(3)

# Prepare the experiment

In [ ]:
EXPERIMENT_NAME = 'adult_census'

In [ ]:
train, test, labels_train, labels_test = train_test_split(data_ohe.drop(columns=['earnings']), data_ohe.earnings, train_size=0.80)

In [ ]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
mlflow.autolog()
with mlflow.start_run(experiment_id=experiment.experiment_id):
    rf = RandomForestClassifier(n_estimators=100, n_jobs=6, max_depth=5)
    rf.fit(train, labels_train)
    yhat_test = rf.predict(test)
    f1 = f1_score(labels_test, yhat_test)

In [37]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
mlflow.autolog()
f1_scoring = make_scorer(f1_score)
with mlflow.start_run(experiment_id=experiment.experiment_id):
    dt = DecisionTreeClassifier(max_depth=10)
    cv_scores = cross_val_score(dt, train, labels_train, cv=5, scoring=f1_scoring, n_jobs=12)
    mlflow.log_metric("CV score", cv_scores)

2022/05/13 21:29:58 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/13 21:29:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [35]:
cv_scores

array([0.67675868, 0.66607222, 0.65047233, 0.65116279, 0.66666667])